In [ ]:
import pandas as pd

df = pd.read_csv("/home/sadid/PycharmProjects/sgs-drawing-analysis/data/miliennium_garage/original4.csv")

df.head()

In [ ]:
pattern = r"^B[-]\d+$"

filtered_df = df[df["value"].astype(str).str.match(pattern, na=False)]
filtered_df.head()

In [ ]:
import cv2
from PIL import Image

image = cv2.imread("/home/sadid/PycharmProjects/sgs-drawing-analysis/data/miliennium_garage/original4.png")

Image.fromarray(image)

In [ ]:
rotated_image = cv2.rotate(image, cv2.ROTATE_90_CLOCKWISE)
Image.fromarray(rotated_image)

In [ ]:
from ocr.ocr_utils import tile_ocr

df_final = tile_ocr(rotated_image, batch_size=24, gpu=True)
df_final.head()

In [ ]:
pattern = r"^B[--]\d+$"

filtered_df = df_final[df_final["value"].astype(str).str.match(pattern, na=False)]
print(filtered_df.shape)
filtered_df.head()

In [ ]:
h, w = rotated_image.shape[:2]

image_copy = rotated_image.copy()
for _, row in filtered_df.iterrows():
    x1 = int(row["x1"] * w)
    y1 = int(row["y1"] * h)
    x2 = int(row["x2"] * w)
    y2 = int(row["y2"] * h)

    cv2.rectangle(image_copy, (x1, y1), (x2, y2), (0, 255, 0), 2)
    cv2.putText(image_copy, str(row["value"]), (x1, y1-5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)

Image.fromarray(image_copy)

In [ ]:
def undo_rotate_90_clockwise(df, r_image):
    h, w = r_image.shape[:2]
    df2 = df.copy()
    df2["x1"] = (df["y1"] * h).astype(int)
    df2["y1"] = ((1 - df["x2"]) * w).astype(int)
    df2["x2"] = (df["y2"] * h).astype(int)
    df2["y2"] = ((1 - df["x1"]) * w).astype(int)
    return df2


df_transformed = undo_rotate_90_clockwise(filtered_df, rotated_image)

h, w = image.shape[:2]

image_copy = image.copy()
for _, row in df_transformed.iterrows():
    x1, y1, x2 , y2 = row["x1"], row["y1"], row["x2"], row["y2"]

    cv2.rectangle(image_copy, (x1, y1), (x2, y2), (0, 255, 0), 2)
    cv2.putText(image_copy, str(row["value"]), (x1, y1-5),
                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)

Image.fromarray(image_copy)